#Sentiment Analysis

###Importing

In [0]:
from collections import Counter
import numpy as np




###Data


In [14]:
# !unzip archive.zip
# %cd archive
# %cd data
# %cd data

/content/archive/data/data


In [0]:
positive_counter = Counter()
negative_counter = Counter()
total_counts = Counter()


data/


##Curating Dataset

In [0]:
def print_review_and_label(i):
  print(labels[i] + "\t:\t" + reviews[i][:80] + "...")
  
g = open('reviews.txt','r')
reviews = list(map(lambda x:x[:-1], g.readlines()))
g.close()

g = open('labels.txt','r')
labels = list(map(lambda x:x[:-1], g.readlines()))
g.close()


In [0]:
print_review_and_label(1)
print_review_and_label(2)
print_review_and_label(3)
print_review_and_label(4)
print_review_and_label(5)
print_review_and_label(6)

In [0]:
for i in range(len(reviews)):
  if(labels[i] == 'positive'):
    for word in reviews[i].split(" "):
      positive_counter[word] += 1
      total_counts[word] += 1
  else:
    for word in reviews[i].split(" "):
      negative_counter[word] += 1
      total_counts[word] += 1

##Create Input/Output Data

In [20]:
vocab = set(total_counts.keys())
vocab_size = len(vocab)
print(vocab_size)

74074


In [21]:
layer_0 = np.zeros((1,vocab_size))
layer_0

array([[0., 0., 0., ..., 0., 0., 0.]])

In [22]:
word2index = {}

for i,word in enumerate(vocab):
  word2index[word] = i


word2index


{'': 0,
 'albinoni': 1,
 'ips': 2,
 'dispatcher': 3,
 'baaaad': 4,
 'peepers': 5,
 'powerweight': 6,
 'dreamy': 7,
 'zerifferelli': 8,
 'gods': 9,
 'prez': 10,
 'message': 11,
 'unconditionally': 12,
 'jarre': 13,
 'newscaster': 14,
 'exterminating': 15,
 'contagonists': 16,
 'conquerer': 17,
 'noticeable': 18,
 'marra': 19,
 'disengaged': 20,
 'imperturbable': 21,
 'colick': 22,
 'kalser': 23,
 'unsuspensful': 24,
 'earner': 25,
 'kazaks': 26,
 'practice': 27,
 'replays': 28,
 'tarantino': 29,
 'mason': 30,
 'doestoevisky': 31,
 'surfeit': 32,
 'cunda': 33,
 'update': 34,
 'hosted': 35,
 'florinda': 36,
 'bountiful': 37,
 'compared': 38,
 'pakeezah': 39,
 'travails': 40,
 'taqueria': 41,
 'combusting': 42,
 'imposition': 43,
 'arabia': 44,
 'crossbeams': 45,
 'lusciously': 46,
 'elbow': 47,
 'assume': 48,
 'puts': 49,
 'unnecessarily': 50,
 'nea': 51,
 'recommendable': 52,
 'flights': 53,
 'conner': 54,
 'frogland': 55,
 'v': 56,
 'gunrunner': 57,
 'khamini': 58,
 'preadolescent': 59,

In [23]:
layer_0.shape

(1, 74074)

'negative'

In [0]:
def update_input_layer(review):
  
  global layer_0
  
  #clear out all previous state, reset the layer to all 0s
  layer_0 *= 0
  for word in review.split(" "):
    layer_0[0][word2index[word]] += 1
    
update_input_layer(reviews[0])    

In [0]:
def get_target_for_label(label):
  if(label == 'positive'):
    return 1
  else:
    return 0
  

In [27]:
for i in range(0,100):
  print(get_target_for_label(labels[i]))

1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0


##Create Neural Network

In [0]:
import time
import sys
import numpy as np

In [0]:
class SentimentNeuralNet:
  
  
  def __init__(self, reviews, labels, hidden_nodes = 10, learning_rate = 0.1):
    
    # Assign a seed to our random number generator to ensure we get reproducable results during development
    np.random.seed(1)
    
    # process the reviews and their associated labels so that everything is ready for training
    self.pre_process_data(reviews, labels)
    
    # Build the network to have the number of hidden nodes and the learning rate that were passed into this initializer. Make the same number of input nodes as
    # there are vocabulary words and create a single output node.
    self.init_network(len(self.review_vocab),hidden_nodes, 1, learning_rate)
   
  
    
  def pre_process_data(self, reviews, labels):
    
    review_vocab = set()            # set does not contain dupliactes
    
    for review in reviews:
      for word in review.split(" "):
        review_vocab.add(word)
    
    # coverting to list for easy iteration with indices
    self.review_vocab = list(review_vocab)      
    
    
    label_vocab = set()
    for label in labels:
      label_vocab.add(label)
    self.label_vocab = list(label_vocab)
    
   
    self.review_vocab_size = len(review_vocab)
    self.review_vocab_size = len(label_vocab)

    self.word2index = {}
    for i, word in enumerate(self.review_vocab):       # enumerate allows us to loop over something and have an automatic counter.
      self.word2index[word] = i

    self.word2index = {}
    for i, label in enumerate(self.label_vocab):       # enumerate allows us to loop over something and have an automatic counter.
      self.word2index[word] = i
      
      
      
      
  def init_network(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
    # Set values
    self.input_nodes = input_nodes
    self.hidden_nodes = hidden_nodes
    self.output_nodes = output_nodes
      
    # Set learning rate
    self.learning_rate = learning_rate
    
    # Initialize weights
    
    # Weights between input layer and hidden layer

    self.weights_0_1 = np.zeros((self.input_nodes, self.hidden_nodes))
    
    # Weights between hidden layer and output layer
    self.weights_1_2 = np.random.normal(0.0, self.output_nodes**-0.5, (self.hidden_nodes, self.output_nodes)) 
    # syntax  numpy.random.normal(loc=0.0, scale=1.0, size=None)
 
    # The input layer, 2 fim matrix, 1 x hidden inputs
    self.layer_1 = np.zeros((1, hidden_nodes))
                            
  def sigmoid(self,x):
    return 1 / (1 + np.exp(-x))
                 
  def sigmoid_output_2_derivative(self,output):
    return output * (1 - output)                        
      
                            
  def get_target_for_label(self,label):
    if(label == 'positive'):
      return 1
    else:
      return 0
    
  
  def train(self, training_reviews_raw, training_labels):
    training_reviews = list()
    for review in training_reviews_raw:
      indices = set()
      for word in review.split(" "):
        if(word in self.word2index.keys()):
          indices.add(self.word2index[word])
          training_reviews.append(list(indices))

    # Number of reviews = Number of Labels
#     assert(len(training_reviews) == len(training_labels))
                            
                            
    # Keep track of correct             
    correct_so_far = 0
             
    # Start time
    start = time.time()
                            
    # Loop over all reviews and labels, run forward and backward pass
    for i in range(len(training_reviews)):
      
      # Get the next review and label
      review = training_reviews[i]
      label = training_labels[i]
      
      ##FORWARD PASS
      
      # Hidden Layer
      # Add in weights for non-zero items
      self.layer_1 *= 0
      for index in review:
        self.layer_1 += self.weights_0_1[index]   
      
      # Output Layer
      layer_2 = self.sigmoid(self.layer_1.dot(self.weights_1_2))
      
      
      ##BACKWARD PASS
      
      # Output error
      layer_2_error = layer_2 - self.get_target_for_label(label)  # Output layer error is the difference between desired target and actual output.
      layer_2_delta = layer_2_error * self.sigmoid_output_2_derivative(layer_2)  # output * (output - 1)
      
      # Backpropagation Error
      layer_1_error = layer_2_delta.dot(self.weights_1_2.T)  # errors propagated to the hidden layer
      layer_1_delta = layer_1_error   # hidden layer gradients - no nonlinearity so it's the same as the error
      
      
      ##UPDATING WEIGHTS
      
      self.weights_1_2 -= self.layer_1.T.dot(layer_2_delta) * self.learning_rate   # update input-to-hidden weights with gradient descent step
      
      # Keep track of correct predictions
      if(layer_2 >= 0.5 and label =='positive'):
        correct_so_far += 1
      elif(layer <= 0.5 and label == 'negative'):
        correct_so_far -= 1
        
      # Prinitng out prediction accuracy, speed throughout the training process
      elapsed_time = float(time.time() - start)
      reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
      
      sys.stdout.write("\rProgress:" + str(100 * i/float(len(training_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct_so_far) + " #Trained:" + str(i+1) \
                             + " Training Accuracy:" + str(correct_so_far * 100 / float(i+1))[:4] + "%")
      
  def test(self, testing_reviews, testing_labels):
    # Calculate accuracy from testing_labels and testing_reviews
    
    # keep track of correct predictions
    correct = 0
    
    # time how many predictions we make per second
    start = time.time()
    
    # Loop through each review and call run to predict corresponding label
    for i in range(le(testing_reviews)):
      pred = self.run(testing_reviews[i])
      if(pred == testing_labels[i]):
        correct+=1
      
      # For debug purposes, print out our prediction accuracy and speed throughout the prediction process.

      elapsed_time = float(time.time() - start)
      reviews_per_second = i / elapsed_time() if elapsed_time > 0 else 0
      
      
      sys.stdout.write("\rProgress:" + str(100 * i/float(len(testing_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct) + " #Tested:" + str(i+1) \
                             + " Testing Accuracy:" + str(correct * 100 / float(i+1))[:4] + "%")
      

  def run(self, review):
    # Return 'positive' or 'negative' prediction for given review
    
    # Run a forward pass like in 'train' function
    
    ## Hidden layer
    
    # Identify the indices in review and add the weights to layer_1
    self.layer_1 *= 0
    unique_indices = set()
    for word in review.lower().split(" "):
      if word in self.word2index.keys():
        unique_indices.add(self.word2index[word])
    for index in unique_indices:
      self.layer_1 += self.weights_0_1[index]

    # Output layer
    layer_2 = self.sigmoid(self.layer_1.dot(self.weights_1_2))
    
    
    # Return Outputs
    if(layer_2[0] >= 0.5):
      return "POSITIVE"
    else:
      return "NEGATIVE"


   
      

In [54]:
mlp = SentimentNeuralNet(reviews[:-1000],labels[:-1000], learning_rate=0.1)
mlp.train(reviews[:-1000],labels[:-1000])

Progress:0.0% Speed(reviews/sec):0.0 #Correct:1 #Trained:1 Training Accuracy:100.%